# Алгоритмы АФП

#### В этой работе мы рассмотрим алгоритмы анализа формальных понятий. Прежде всего нас будут интересовать алгоритмы построения алгебраической решетки. Начать стоит с обсуждения наивного алгоритма, то есть самого простого и неэффективного алгоритма. В таком алгоритме нам необходимо пройтись по всем подмножествам множества объектов нашего формального контеста $K = (G, M, I)$, и для каждого подмножества построить его замыкание, если оно совпадает с самим подмножеством, то оно составляет понятие. Далее, упорядочивая подмножества, можно построить все понятия и граф, который свяжет эти понятия и образует необходимую решетку. Реализуем данный алгоритм:

In [ ]:
import numpy as np


def f(s):
    global ans_graph, repeat, graph
    if s in repeat:
        ans_graph = 0
        return
    for g in graph:
        if g[0] == s:
            f(g[1])
    return


n, m = map(int, input().split())
mas = np.empty((n, m), dtype=int)
for i in range(n):
    mas[i] = np.array(list(map(int, input().split())))
lattice = list([(np.array([], dtype=int), np.array([1 for i in range(m)]))])
graph = []
hypotheses = np.empty((0, m))
ans_graph = 0
current = 1


for i in range(1, 1 << n):
    vector = np.ones(m, dtype=int)
    arr = np.array([], dtype=int)
    mis = np.array([], dtype=int)
    y = 1
    for j in range(n):
        if i % (2*y) // y:
            vector = np.bitwise_and(vector, mas[j])
            arr = np.append(arr, j)
        y *= 2
    for j in range(n):
        if (np.bitwise_and(vector, mas[j]) == vector).all():
            mis = np.append(mis, j)
    y = 0
    if np.array_equal(mis, arr):
        repeat = set()
        for j in range(len(lattice) - 1, -1, -1):
            if set(lattice[j][0]) <= set(arr):
                ans_graph = 1
                f(j)
                yy = ans_graph
                if yy:
                    graph.append((j, current))
                    repeat.add(j)
                    y = 1
        if y:
            lattice.append((arr, vector))
            current += 1


#### Так как мы проходимся по всем подмножествам множества объектов, а внутри этого цикла также проходимся по всей объектам и признакам, а для строительства графа в худшем случае проходимся двойным циклом по всем построенным понятиям, то временная сложность алгоритма $O(2^{|G|}|G||M||L|^{2})$. Заметим, что если объектов больше признаков, то выгодно поменять местами объекты и признаки, таким образом мы можем существенно выиграть во времени, уменьшив главный показательный множитель. Проведем эксперимент: посмотрим, как время выполнения алгоритма зависит от входных параметров, для этого будем использовать случайно сгенерированную матрицу. Результаты в таблице ниже:

In [10]:
import pandas as pd
example1 = pd.DataFrame({'$n$': [7, 10, 11, 12, 13, 14, 15], '$m$': [7, 10, 11, 12, 13, 14, 15], 'время выполнения (сек)': [0.031, 0.360, 0.719, 1.899, 8.963, 15.837, 42.443]})
example1

,$n$,$m$,время выполнения (сек)
0,7,7,0.031
1,10,10,0.360
2,11,11,0.719
3,12,12,1.899
4,13,13,8.963
5,14,14,15.837
6,15,15,42.443


#### Как видно из таблицы, увеличение числа объектов на один увеличивает в два раза время выполнения алгоритма, что неудивительно, ведь у нас показательная скорость работы алгоритма. Результаты показывают, что данный алгоритм неспособен работать даже с небольшими данными, поэтому нам необходимы другие алгоритмы, которые будут работать гораздо быстрее. Основная проблема данного алгоритма в том, что нам необходимо проходить все подмножества множества объектов, это дает нам как минимум показательную скорость работы алгоритма.

#### Возникает вопрос: действительно ли необходимо проходить все подмножества множества объектов или же достаточно пройти лишь некоторые, чтобы полностью построить решетку? Оказывается, проходить все подмножества не обязательно. Для начала необходимо упорядочить признаки. Дальше можно воспользоваться теоремой, которая утверждает, что если нам дано некоторое множество $A$ признаков, то следующее по лексикографическому порядку множество признаков, которое будет образовывать понятие, считается по формуле $A \oplus m$, где $m$ определяется как наибольший признак, для которого $A < A \oplus m$, причем сравнение идет только для признаков, меньших или равных $m$, а все признаки, бОльшие $m$, игнорируются. Выражение $A \oplus m$ означает, что все признаки, большие $m$, удаляются из множества $A$, а сам признак $m$ добавляется в множество $A$. На этой теореме основан алгоритм NextClosure. Сначала он считает замыкание пустого множества признаков. Далее он находит такой $m$, который удовлятворяет условиям выше, и с его помощью строится следующее по лексикографическому порядку понятие. Потом мы снова находим $m$ и снова строим понятие, и так до тех пор, пока мы не получим понятие, содержащее все признаки, то есть понятие, которое находится на верхушке решетки. На каждом шаге итерации мы получаем новое понятие и сохраняем его. Такой алгоритм позволяет построить все понятия, но с его помощью невозможно построить решетку, так как мы не знаем, как взаимосвязаны понятия. Реализуем данный алгоритм: 

In [ ]:
import numpy as np


def closure(make_closure):
    global n, m, mas
    ans_intent = np.array([1 for i in range(m)], dtype=int)
    for i in range(n):
        if (mas[i] == np.bitwise_or(mas[i], make_closure[1])).all():
            ans_intent = np.bitwise_and(ans_intent, mas[i])
    ans = [make_closure[0], ans_intent]
    return ans


def extent(concept):
    global m, mas
    ans_extent = np.array([1 for i in range(n)])
    for i in range(m):
        if concept[1][i]:
            ans_extent = np.bitwise_and(ans_extent, mas[:, i])
    concept[0] = ans_extent
    return concept


n, m = map(int, input().split())
mas = np.empty((n, m), dtype=int)
for i in range(n):
    mas[i] = np.array(list(map(int, input().split())))
current = [np.array([], dtype=int), np.array([0 for i in range(m)], dtype=int)]
flag = 1

current = closure(current)
extent(current)
print(current)
while flag:
    for i in range(m - 1, -1, -1):
        if current[1][i]:
            current[1][i] = 0
        else:
            current[1][i] = 1
            new_concept = closure(current)
            current[1][i] = 0
            elem_less = 1
            for j in range(m):
                if current[1][j] and not new_concept[1][j]:
                    elem_less = 0
            if elem_less:
                current = new_concept
                extent(current)
                print(current)
                break
    if np.count_nonzero(current[1] == 1) == m:
        flag = 0

#### Исследуем временную сложность алгоритма. Обратим внимание на то, что самая времязатратная часть алгоритма - это построение замыкания, то есть построение для некоторого множества объектов $A$ множества объектов $A''$. Для построения мы проходимся по всем объектам и отмечаем, какие признаки присущи данному объекту, обрабатывая эти данные специальным способом (функция closure в коде выше). Таким образом, временная сложность построения замыкания равна $O(|G||M|)$. Теперь посмотрим на алгоритм построения одного понятия, в нем мы проходимся в обратном порядке по всем признакам, пока не найдем нужный нам и не обработаем его нужным нам образом. То есть временная сложность дополнительно умножается на $|M|$. Для построения решетки мы должны построить все понятия, то есть сложность также умножается на $|L|$, то есть на количество понятий в нашей решетки. Суммарно, временная сложность алгоритма равна $O(|G||M^{2}||L|)$. 

#### Проведем эксперимент: исследуем, как скокорость выполнения алгоритма зависит от входных параметров. Так как скорость зависит от входных данных в том числе и при одинаковых количествах объектов и признаков, то будем проводить несколько экспериментов. Результат расположен в таблице ниже:

In [8]:
import pandas as pd
example2 = pd.DataFrame({'$n$': [10, 20, 50, 100, 500, 1000, 2000, 10, 1000], '$m$': [10, 20, 50, 100, 500, 1000, 2000, 1000, 10], 'время выполнения (сек), первый эксперимент': [0.006, 0.009, 0.053, 0.046, 1.791, 4.302, 9.704, 0.026, 7.942], 'время выполнения (сек), второй эксперимент': [0.003, 0.009, 0.098, 0.109, 1.120, 3.878, 14.727, 0.033, 5.760], 'время выполнения (сек), третий эксперимент': [0.004, 0.011, 0.034, 0.066, 1.024, 5.258, 5.320, 0.037, 13.322]})
example2

,$n$,$m$,"время выполнения (сек), первый эксперимент","время выполнения (сек), второй эксперимент","время выполнения (сек), третий эксперимент"
0,10,10,0.006,0.003,0.004
1,20,20,0.009,0.009,0.011
2,50,50,0.053,0.098,0.034
3,100,100,0.046,0.109,0.066
4,500,500,1.791,1.120,1.024
5,1000,1000,4.302,3.878,5.258
6,2000,2000,9.704,14.727,5.320
7,10,1000,0.026,0.033,0.037
8,1000,10,7.942,5.760,13.322


#### Можно заметить, что несмотря на одинаковые $n$ и $m$, время работы алгоритма может отличаться в несколько раз. Связано это с тем, что время работы алгоритма прямо пропорционально зависит от числа понятий, и заранее мы не можем сказать, как много понятий у нас будет. Из-за этого в худшем случае временная сложность алгоритма экспоненциальная, потому что количество понятий может экспоненциально зависеть от числа объектов или признаков. К примеру, пусть и в качестве объектов, и в качестве признаков будут натуральные числа, не превосходящие $n$, и объект обладает признаком, если число-объект не равен числу-признаку. Тогда каждое множество объектов составляет понятие, и всего понятий $2^{n}$. Это является проблемой всех алгоритмов анализа формальных понятий, так как в любом случае нам необходимо построить все понятия, даже если это занимает экспоненциальное время, и в худшем случае любой алгоритм АФП будет работать за как минимум экспоненциальное время. Также заметим, что в нашем случае сложность зависит от мощности множества объектов и от квадрата мощности множества признаков. Как видно из 7 и 8 столбцов из таблицы выше, это может играть большую роль, и поменяв местами объекты и признаки, мы можем очень сильно выиграть во времени. 

#### Теперь рассмотрим другой алгоритм: пусть у нас есть уже построенное множество понятий $C$, и мы добавляем в множество объектов один объект $g$. Перед нами стоит цель перестроить решетку, очевидно, что строить решетку с нуля очень неэффективно, необходимо использовать уже построенное множество понятий. Итак, пусть мы хотим построить новое множество понятий $C_g$, и в него входит некоторое понятие $(A, B)$, тогда существует три случая: если $g$ не входит в $A$, тогда мы просто берем понятие из уже построенной решетки, если $g$ входит в $A$ и $(A\setminus\{g\} , B)$ входит в $C$, тогда в $C_g$ добавляем $(A\cup\{g\}, B)$, и если $g$ входит в $C$, но $(C\setminus\{g\}, D)$ не входит в $C$, тогда добавляем $(C\setminus\{g\}, (C\setminus\{g\})')$ в $C_g$. Алгоритм реализован ниже:

In [ ]:
import numpy as np


n, m, num = map(int, input().split())
mas = np.empty((n, m), dtype=int)
for i in range(n - 1):
    mas[i] = np.array(list(map(int, input().split())))
concepts = list([(np.array([0 for i in range(n)]), np.array([1 for i in range(m)]))])
for i in range(num - 1):
    extent = np.array(list(map(int, input().split())))
    intent = np.array(list(map(int, input().split())))
    concepts.append((extent, intent))
mas[-1] = np.array(list(map(int, input().split())))

for i in concepts:
    if (np.bitwise_or(i[1], mas[-1]) == mas[-1]).all():
        i[0][-1] = 1
        print(i[0], i[1])
        i[0][-1] = 0
    else:
        print(i[0], i[1])
        intent = np.array(np.bitwise_and(i[1], mas[-1]))
        extent = np.array([0 for j in range(n)])
        for j in range(n):
            if (np.bitwise_and(mas[j], intent) == intent).all():
                extent[j] = 1
        print(extent, intent)


#### Заметим, что с помощью этого алгоритма можно построить алгоритм, который строит всю решетку. Для этого мы будем постепенно вводить по одному объекту, и каждый раз при вводе нового объекта будем перестраивать решетку. Тем самым к последнему объекту мы получим полную решетку. Реализуем этот алгоритм:

In [ ]:
import numpy as np


n, m = map(int, input().split())
mas = np.empty((n, m), dtype=int)
for i in range(n):
    mas[i] = np.array(list(map(int, input().split())))
concepts = list([(np.array([], dtype=int), np.array([1 for i in range(m)]))])


for i in range(n):
    new = list([(np.array([], dtype=int), np.array([1 for j in range(m)]))])
    for conc in concepts:
        if (np.bitwise_or(conc[1], mas[i]) == mas[i]).all():
            conc[0][-1] = 1
            new.append((conc[0], conc[1]))
            conc[0][-1] = 0
        else:
            new.append((conc[0], conc[1]))
            intent = np.array(np.bitwise_and(conc[1], mas[i]))
            extent = np.array([0 for j in range(n)])
            for j in range(n):
                if (np.bitwise_and(mas[j], intent) == intent).all():
                    extent[j] = 1
            new.append((extent, intent))
    concepts = new


#### Данный алгоритм называется алгоритмом Норриса. Теперь также проведем эксперимент и измерим скорость выполнения алгоритма в зависимости от входных данных:

In [7]:
import pandas as pd
example3 = pd.DataFrame({'$n$': [10, 15, 20], '$m$': [10, 15, 20], 'время выполнения (сек), первый эксперимент': [0.047, 0.640, 7.391], 'время выполнения (сек), второй эксперимент': [0.031, 0.766, 5.727], 'время выполнения (сек), третий эксперимент': [0.047, 0.230, 9.747]})
example3

,$n$,$m$,"время выполнения (сек), первый эксперимент","время выполнения (сек), второй эксперимент","время выполнения (сек), третий эксперимент"
0,10,10,0.047,0.031,0.047
1,15,15,0.640,0.766,0.230
2,20,20,7.391,5.727,9.747


#### Алгоритмическая сложность алгоритма такая же, как и у предыдущего, $O(|G|^{2}|M||L|)$, так как мы обновляем решетку для каждого объекта, и для обновления решетки нам необходимо пройтись по всем понятиям, и внутри этого цикла пройтись по всем объектам и признакам. Несмотря на то, что временная сложность этого и предыдущего алгоритмов одинаковые, этот алгоритм работает куда медленнее. Это связано с тем, что при обновлении решетки получается много одинаковых понятий, которые мы никак не убираем, тем самым множитель $|L|$ увеличивается с увеличением числа объектов.

#### Подведем итоги: лучшей временной сложностью, с которой можно построить все понятия решетки, является $O(|G|^{2}|M||L|)$ или же $O(|G||M|^{2}|L|)$, и достигнуть ее помогает алгоритм NextClosure, алгоритм Норриса дает более худшие результаты, однако он тоже может быть эффективным при решении проблемы генерации однотипных понятий. Также отметим, что данные алгоритмы только строят все понятия, не выявляя связи между ними и не строя графа, однако эта задача является куда более простой, чем построение понятий, и занимает меньше времени. 